# Model Deployment
## Azure ML and MLFlow tracking

In [5]:
# Deploy model
# Connect to workspace
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()
ml_client = MLClient(
    credential=credential,
    subscription_id="49b5441f-dda4-47a9-81c4-13272430f4ff",
    resource_group_name="rg-pooya120-dev",
    workspace_name="amlwpooya120dev",
)

In [6]:
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
import os

score_dir = './score'
os.makedirs(score_dir, exist_ok=True)

In [15]:
%%writefile {score_dir}/score.py
import os
import logging
import json
import numpy
import joblib
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import requests
from datetime import datetime
FN_URL = 'https://fnapppooya120dev.azurewebsites.net/api'

def init():
    """
    This function is called when the container is initialized/started, typically after create/update of the deployment.
    You can write the logic here to perform init operations like caching the model in memory
    """
    global model
    # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    model_path = os.path.join(
        os.getenv("AZUREML_MODEL_DIR"), "model/model.pkl"
    )
    print(model_path)
    logging.info("Loading model from "+model_path)
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)
    logging.info("Init complete")


def run(raw_data):
    """
    This function is called for every invocation of the endpoint to perform the actual scoring/prediction.
    In the example we extract the data from the json input and call the scikit-learn model's predict()
    method and return the result back
    """
    logging.info("Request received")
    arr = json.loads(raw_data)
    payload = []
    results = []
    for data in arr:
        id = data['id']
        ts = data['ts']
        inputs = data['inputs']
        npdata = numpy.array(inputs)
        value = model.predict(npdata)[0]
        results.append(value)
        payload.append({"id":id,"ts":ts, "inputs":json.dumps(inputs), "value":value, "type":"prediction"})
    logging.info("Request processed")
    requests.post(FN_URL+'/DataLogger',json=payload)
    return results


Overwriting ./score/score.py


In [16]:
%%writefile {score_dir}/conda.yml
name: model-env
channels:
  - conda-forge
dependencies:
  - python=3.8
  - numpy=1.21.2
  - pip=21.2.4
  - scikit-learn=0.24.2
  - scipy=1.7.1
  - pandas>=1.1,<1.2
  - pip:
    - azureml-defaults>=1.42.0
    - azureml-inference-server-http
    - inference-schema[numpy-support]==1.3.0
    - xlrd==2.0.1
    - mlflow== 1.26.1
    - azureml-mlflow==1.42.0
    - psutil>=5.8,<5.9
    - tqdm>=4.59,<4.60
    - ipykernel~=6.0
    - matplotlib

Overwriting ./score/conda.yml


In [10]:
from azure.ai.ml.entities import Environment

custom_env_name = "bike-share-env"

env = Environment(
    name=custom_env_name,
    description="Custom environment for Bike Share pipeline",
    tags={"scikit-learn": "0.24.2"},
    conda_file= "score/conda.yml",
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:latest",
)
env = ml_client.environments.create_or_update(env)

print(
    f"Environment {env.name} registered to workspace. "
    f"Environment version is {env.version}."
)

Environment bike-share-env registered to workspace. Environment version is 4.


## Endpoints and Deployments

- An Endpoint is an HTTP server that allows interaction with the model(s).
- An endpoint may serve multiple models.
- Each model must be deployed in order for the endpoint to serve it.
- Hence, one Endpoint can have many Deployments.
- Blue-Green deployment and traffic management can be handled here or using az CLI.

In [11]:
# Create an endpoint name
endpoint_name = "my-endpoint3"

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name = endpoint_name, 
    description="Bike share endpoint",
    auth_mode="key",
)

ml_client.online_endpoints.begin_create_or_update(endpoint, ).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://my-endpoint3.westus.inference.ml.azure.com/score', 'openapi_uri': 'https://my-endpoint3.westus.inference.ml.azure.com/swagger.json', 'name': 'my-endpoint3', 'description': 'Bike share endpoint', 'tags': {}, 'properties': {'azureml.onlineendpointid': '/subscriptions/49b5441f-dda4-47a9-81c4-13272430f4ff/resourcegroups/rg-pooya120-dev/providers/microsoft.machinelearningservices/workspaces/amlwpooya120dev/onlineendpoints/my-endpoint3', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/49b5441f-dda4-47a9-81c4-13272430f4ff/providers/Microsoft.MachineLearningServices/locations/westus/mfeOperationsStatus/oe:130523ce-1b7a-4fb0-a10c-81b9b91961ad:24fa020a-ba28-46b4-a7a7-510072167edb?api-version=2022-02-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/49b5441f-dda4-47a9-81c4-13272430f4ff/resourceGroups/rg-pooya120-dev/providers/Microsoft.MachineLearnin

In [13]:
# grab the model to be deployed and the serving environment
model = ml_client.models.get('sklearn.ensemble._forest.RandomForestRegressor',4)
env = ml_client.environments.get('bike-share-env', 4)

In [17]:

deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="./score", scoring_script="score.py"
    ),
    instance_type="Standard_DS3_v2",
    instance_count=1,
    app_insights_enabled=True,
)
ml_client.online_deployments.begin_create_or_update(deployment=deployment, ).result()

Check: endpoint my-endpoint3 exists
Uploading score (0.0 MBs): 100%|██████████| 2648/2648 [00:00<00:00, 55913.64it/s]


data_collector is not a known attribute of class <class 'azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.ManagedOnlineDeployment'> and will be ignored


................................

ManagedOnlineDeployment({'private_network_connection': False, 'provisioning_state': 'Succeeded', 'data_collector': None, 'endpoint_name': 'my-endpoint3', 'type': 'Managed', 'name': 'blue', 'description': None, 'tags': {}, 'properties': {'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/49b5441f-dda4-47a9-81c4-13272430f4ff/providers/Microsoft.MachineLearningServices/locations/westus/mfeOperationsStatus/od:130523ce-1b7a-4fb0-a10c-81b9b91961ad:3ea74fa6-9965-45af-bd31-22e49dc08bbd?api-version=2022-02-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/49b5441f-dda4-47a9-81c4-13272430f4ff/resourceGroups/rg-pooya120-dev/providers/Microsoft.MachineLearningServices/workspaces/amlwpooya120dev/onlineEndpoints/my-endpoint3/deployments/blue', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/pooyaalavian1/code/Users/pooyaalavian/experimentation/ds', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object a